In [1]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import collections
import csv
import random
import pickle
from torch.utils.data import DataLoader
import dgl
import pandas as pd

class Subgraphs(Dataset):


    def __init__(self, root, mode, subgraph_list, subgraph2label, subgraph2center_node):   

        self.subgraph2label = subgraph2label
        self.subgraph_list = subgraph_list
        self.subgraph2center_node = subgraph2center_node
        
        self.data = pd.read_csv(os.path.join(root, mode + '.csv'))  # csv path
    
    def __getitem__(self, index):
        
        return self.subgraph_list[self.data.iloc[index]['name']], self.subgraph2label[self.data.iloc[index]['name']], self.subgraph2center_node[self.data.iloc[index]['name']] 

    def __len__(self):
        # as we have built up to batchsz of sets, you can sample some small batch size of sets.
        return len(self.data)
    
def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels, center_nodes = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.LongTensor(labels), torch.LongTensor(center_nodes)


In [2]:
class Subgraphs_test(Dataset):


    def __init__(self, root, mode, subgraph_list, subgraph2label, subgraph2center_node, k_shot, n_qry):   

        self.subgraph2label = subgraph2label
        self.subgraph_list = subgraph_list
        self.subgraph2center_node = subgraph2center_node
        
        self.data = pd.read_csv(os.path.join(root, mode + '.csv'))  # csv path
        self.k_shot = k_shot
        self.n_qry = n_qry
        
        self.labels = np.unique(self.data.label.values)
        self.labels_dict = dict(zip(list(self.labels), list(range(len(self.labels)))))
        
    def __getitem__(self, index):
        support = []
        query = []
        
        for i in self.labels:
            df_labels = self.data[self.data.label == i].reset_index(drop = True)
            support = support + list(df_labels.sample(n = self.k_shot)['name'].values)
            query = query + list(df_labels[~df_labels.name.isin(support)].sample(n = self.n_qry)['name'].values)
        
        return [self.subgraph_list[i] for i in support], [self.labels_dict[self.subgraph2label[i]] for i in support], [self.subgraph2center_node[i] for i in support], [self.subgraph_list[i] for i in query], [self.labels_dict[self.subgraph2label[i]] for i in query], [self.subgraph2center_node[i] for i in query]

    def __len__(self):
        # 100 "batches"
        return 100
    
def collate_test(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    
    graphs, labels, center_nodes, g_qry, l_qry, c_qry = map(list, zip(*samples))
    batched_graph_spt = dgl.batch(graphs[0])
    batched_graph_qry = dgl.batch(g_qry[0])
    
    return batched_graph_spt, torch.LongTensor(labels), torch.LongTensor(center_nodes), batched_graph_qry, torch.LongTensor(l_qry), torch.LongTensor(c_qry)


In [3]:
import dgl.function as fn
import torch
import torch.nn as nn


# Sends a message of node feature h.
msg = fn.copy_src(src='h', out='m')

def reduce(nodes):
    """Take an average over all neighbor node features hu and use it to
    overwrite the original node feature."""
    accum = torch.mean(nodes.mailbox['m'], 1)
    return {'h': accum}

class NodeApplyModule(nn.Module):
    """Update the node feature hv with ReLU(Whv+b)."""
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        # Initialize the node features with h.
        g.ndata['h'] = feature
        g.update_all(msg, reduce)
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [4]:
import copy

In [5]:
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()

        self.layers = nn.ModuleList([
            GCN(in_dim, hidden_dim, F.relu),
            GCN(hidden_dim, hidden_dim, F.relu)])
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g, to_fetch, features):
        # For undirected graphs, in_degree is the same as
        # out_degree.
        h = g.in_degrees().view(-1, 1).float().to(device)
        #h = torch.tensor([1.]*g.number_of_nodes()).view(-1, 1).float()

        #h = features.float()
        for conv in self.layers:
            h = conv(g, h)
        g.ndata['h'] = h
        #print(h.shape)
        #hg = dgl.mean_nodes(g, 'h')
        #print(to_fetch)
        num_nodes_ = g.batch_num_nodes
        num_nodes_.insert(0, 0)
        offset = torch.cumsum(torch.LongTensor(num_nodes_), dim = 0)[:-1].to(device)
        hg = h[to_fetch + offset]
        #print(hg.shape)
        #print(hg.shape)
        #print(h[0].shape)
        #hg = h[g.nodes[0].data['center_node'].detach().numpy()[0]]
        #print(hg.shape)
        return hg, self.classify(hg)

In [6]:
from torch.utils.data import DataLoader
import torch.optim as optim
path = './data/single_graph/cycle_random1000/META_SETUP_LABEL/'
with open(path + 'list_subgraph.pkl', 'rb') as f:
    total_subgraph = pickle.load(f)
    
with open(path + 'label.pkl', 'rb') as f:
    info = pickle.load(f)

with open(path + 'center.pkl', 'rb') as f:
    center_node = pickle.load(f)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
class Classifier_Finetune(nn.Module):
    def __init__(self, hidden_dim, n_classes):
        super(Classifier_Finetune, self).__init__()

        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, model, g, to_fetch, features):
        
        h, _ = model(g, to_fetch, features)
        
        return self.classify(h)

In [8]:
def run(fold_n):
    path = './data/single_graph/cycle_random1000/META_SETUP_LABEL/'

    path = path + 'fold'+str(fold_n)+'/'
    trainset = Subgraphs(path, 'train', total_subgraph, info, center_node)
    valset = Subgraphs(path, 'val', total_subgraph, info, center_node)
    testset = Subgraphs_test(path, 'test', total_subgraph, info, center_node, 1, 12)

    data_loader_test = DataLoader(testset, batch_size=1, shuffle=True, collate_fn=collate_test)

    data_loader = DataLoader(trainset, batch_size=64, shuffle=True, collate_fn=collate)
    
    model = Classifier(1, 256, max(info.values()) + 1)
    model.to(device)
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    model.train()
    
    print('model training')
    for epoch in range(50):
        epoch_loss = 0
        for iter, (bg, label, to_fetch) in enumerate(data_loader):
            bg = bg.to(device)
            label = label.to(device)
            features = 1
            hid, prediction = model(bg, to_fetch, features)
            #print(prediction.shape)
            loss = loss_func(prediction, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.detach().item()
        epoch_loss /= (iter + 1)
        
        if epoch%25 ==0:
            print('Epoch {}, training loss {:.4f}'.format(epoch, epoch_loss))
    
    print('model finetuning')
    accs = []
    for iter, (bg, label, to_fetch, bg_qry, label_qry, to_fetch_qry) in enumerate(data_loader_test):
        model_finetune = Classifier_Finetune(256, 2)
        to_fetch = torch.LongTensor(to_fetch.reshape(-1,))
        label = label.reshape(-1,)

        to_fetch_qry = torch.LongTensor(to_fetch_qry.reshape(-1,))
        label_qry = label_qry.reshape(-1,)    

        update_step = 10
        loss_steps = []
        acc_steps = []

        for i in range(update_step):
            model_finetune.train()
            bg_ = copy.deepcopy(bg)
            prediction = model_finetune(model, bg_, to_fetch, 1)
            loss = loss_func(prediction, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            model_finetune.eval()
            bg_ = copy.deepcopy(bg_qry)
            pred_qry = model_finetune(model, bg_, to_fetch_qry, 1)
            probs_Y = torch.softmax(pred_qry, 1)
            argmax_Y = torch.max(probs_Y, 1)[1].view(-1, 1)

            label_q = torch.tensor(label_qry).float().view(-1, 1)
            acc = (label_q == argmax_Y.detach().cpu().float()).sum().item() / len(label_q) * 100

            acc_steps.append(acc)
            loss_steps.append(loss.item())
        accs.append(acc_steps)
        if iter%20 == 0:
            print('step ', iter)
    print(np.mean(np.array(accs), 0))
    
    
    print('model KNN')
    accs = []
    for iter, (bg, label, to_fetch, bg_qry, label_qry, to_fetch_qry) in enumerate(data_loader_test):
        to_fetch = torch.LongTensor(to_fetch.reshape(-1,))
        label = label.reshape(-1,)
        
        to_fetch_qry = torch.LongTensor(to_fetch_qry.reshape(-1,))
        label_qry = label_qry.reshape(-1,) 
        
        bg_ = copy.deepcopy(bg)
        h, prediction = model(bg_, to_fetch, 1)
        
        bg_ = copy.deepcopy(bg_qry)
        h_qry, pred_qry = model(bg_, to_fetch_qry, 1)
        
        from sklearn.metrics import pairwise_distances
        
        dist = pairwise_distances(h.detach().numpy(), h_qry.detach().numpy())
        
        y_pred = np.argmin(dist, 0).reshape(-1,)
        
        label_dict = dict(zip(label.numpy(), list(range(label.numpy().reshape(-1,).shape[0]))))
        y_pred = np.array([label_dict[i] for i in y_pred])
        #print(y_pred.shape)
        label_q = label_qry.float().view(-1, ).numpy()

        acc = sum(label_q == y_pred) / len(label_q) * 100

        accs.append(acc)
        
        if iter%20 == 0:
            print('step ', iter)
            
    print(np.mean(np.array(accs)))

In [9]:
run(1)

model training
Epoch 0, training loss 1.7086
Epoch 25, training loss 0.7484
model finetuning


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  0
step  20
step  40
step  60
step  80
[52.75       52.95833333 53.08333333 54.25       55.66666667 57.75
 60.66666667 62.91666667 65.125      66.875     ]
model KNN
step  0
step  20
step  40
step  60
step  80
86.16666666666666


In [10]:
run(2)

model training
Epoch 0, training loss 1.7894
Epoch 25, training loss 0.8364
model finetuning


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  0
step  20
step  40
step  60
step  80
[52.70833333 53.75       54.95833333 57.70833333 60.625      62.20833333
 66.375      69.66666667 72.04166667 73.91666667]
model KNN
step  0
step  20
step  40
step  60
step  80
99.25


In [11]:
run(3)

model training
Epoch 0, training loss 1.7392
Epoch 25, training loss 0.8194
model finetuning
step  0


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  20
step  40
step  60
step  80
[46.875      47.70833333 49.08333333 49.91666667 52.08333333 54.70833333
 57.29166667 59.625      61.79166667 63.375     ]
model KNN
step  0
step  20
step  40
step  60
step  80
88.79166666666666


In [12]:
run(4)

model training
Epoch 0, training loss 1.7560
Epoch 25, training loss 0.8165
model finetuning
step  0


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  20
step  40
step  60
step  80
[47.29166667 47.83333333 48.91666667 50.33333333 52.16666667 54.29166667
 57.         59.375      62.45833333 65.20833333]
model KNN
step  0
step  20
step  40
step  60
step  80
86.79166666666664


In [13]:
run(5)

model training
Epoch 0, training loss 1.7770
Epoch 25, training loss 0.8516
model finetuning
step  0


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  20
step  40
step  60
step  80
[51.04166667 51.54166667 52.45833333 54.66666667 56.75       59.625
 62.41666667 65.58333333 68.         70.5       ]
model KNN
step  0
step  20
step  40
step  60
step  80
97.83333333333331


In [19]:
def run_BA(fold_n):
    path = './data/single_graph/BA/META_SETUP_LABEL/'
    
    with open(path + 'list_subgraph.pkl', 'rb') as f:
        total_subgraph = pickle.load(f)
    
    with open(path + 'label.pkl', 'rb') as f:
        info = pickle.load(f)

    with open(path + 'center.pkl', 'rb') as f:
        center_node = pickle.load(f)
    
    path = path + 'fold'+str(fold_n)+'/'
    trainset = Subgraphs(path, 'train', total_subgraph, info, center_node)
    valset = Subgraphs(path, 'val', total_subgraph, info, center_node)
    testset = Subgraphs_test(path, 'test', total_subgraph, info, center_node, 1, 6)

    data_loader_test = DataLoader(testset, batch_size=1, shuffle=True, collate_fn=collate_test)

    data_loader = DataLoader(trainset, batch_size=64, shuffle=True, collate_fn=collate)
    
    model = Classifier(1, 256, max(info.values()) + 1)
    model.to(device)
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    model.train()
    
    print('model training')
    for epoch in range(100):
        epoch_loss = 0
        for iter, (bg, label, to_fetch) in enumerate(data_loader):
            bg = bg.to(device)
            label = label.to(device)
            features = 1
            hid, prediction = model(bg, to_fetch, features)
            #print(prediction.shape)
            loss = loss_func(prediction, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.detach().item()
        epoch_loss /= (iter + 1)
        
        if epoch%25 ==0:
            print('Epoch {}, training loss {:.4f}'.format(epoch, epoch_loss))
    
    print('model finetuning')
    accs = []
    for iter, (bg, label, to_fetch, bg_qry, label_qry, to_fetch_qry) in enumerate(data_loader_test):
        model_finetune = Classifier_Finetune(256, 2)
        to_fetch = torch.LongTensor(to_fetch.reshape(-1,))
        label = label.reshape(-1,)

        to_fetch_qry = torch.LongTensor(to_fetch_qry.reshape(-1,))
        label_qry = label_qry.reshape(-1,)    

        update_step = 10
        loss_steps = []
        acc_steps = []

        for i in range(update_step):
            model_finetune.train()
            bg_ = copy.deepcopy(bg)
            prediction = model_finetune(model, bg_, to_fetch, 1)
            loss = loss_func(prediction, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            model_finetune.eval()
            bg_ = copy.deepcopy(bg_qry)
            pred_qry = model_finetune(model, bg_, to_fetch_qry, 1)
            probs_Y = torch.softmax(pred_qry, 1)
            argmax_Y = torch.max(probs_Y, 1)[1].view(-1, 1)

            label_q = torch.tensor(label_qry).float().view(-1, 1)
            acc = (label_q == argmax_Y.detach().cpu().float()).sum().item() / len(label_q) * 100

            acc_steps.append(acc)
            loss_steps.append(loss.item())
        accs.append(acc_steps)
        if iter%20 == 0:
            print('step ', iter)
    print(np.mean(np.array(accs), 0))
    
    
    print('model KNN')
    accs = []
    for iter, (bg, label, to_fetch, bg_qry, label_qry, to_fetch_qry) in enumerate(data_loader_test):
        to_fetch = torch.LongTensor(to_fetch.reshape(-1,))
        label = label.reshape(-1,)
        
        to_fetch_qry = torch.LongTensor(to_fetch_qry.reshape(-1,))
        label_qry = label_qry.reshape(-1,) 
        
        bg_ = copy.deepcopy(bg)
        h, prediction = model(bg_, to_fetch, 1)
        
        bg_ = copy.deepcopy(bg_qry)
        h_qry, pred_qry = model(bg_, to_fetch_qry, 1)
        
        from sklearn.metrics import pairwise_distances
        
        dist = pairwise_distances(h.detach().numpy(), h_qry.detach().numpy())
        
        y_pred = np.argmin(dist, 0).reshape(-1,)
        
        label_dict = dict(zip(label.numpy(), list(range(label.numpy().reshape(-1,).shape[0]))))
        y_pred = np.array([label_dict[i] for i in y_pred])
        #print(y_pred.shape)
        label_q = label_qry.float().view(-1, ).numpy()

        acc = sum(label_q == y_pred) / len(label_q) * 100

        accs.append(acc)
        
        if iter%20 == 0:
            print('step ', iter)
            
    print(np.mean(np.array(accs)))

In [20]:
run_BA(1)

model training
Epoch 0, training loss 2.4665
Epoch 25, training loss 0.9601
Epoch 50, training loss 0.9331
Epoch 75, training loss 0.8742
model finetuning


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  0
step  20
step  40
step  60
step  80
[52.16666667 53.5        54.5        56.25       59.         63.91666667
 65.75       67.25       68.91666667 70.        ]
model KNN
step  0
step  20
step  40
step  60
step  80
92.49999999999999


In [21]:
run_BA(2)

model training
Epoch 0, training loss 3.2333
Epoch 25, training loss 1.4629
Epoch 50, training loss 1.3171
Epoch 75, training loss 1.0693
model finetuning


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  0
step  20
step  40
step  60
step  80
[49.5        49.83333333 50.91666667 52.91666667 54.16666667 54.91666667
 56.5        59.16666667 61.16666667 62.66666667]
model KNN
step  0
step  20
step  40
step  60
step  80
89.66666666666664


In [22]:
run_BA(3)

model training
Epoch 0, training loss 2.8521
Epoch 25, training loss 1.4942
Epoch 50, training loss 1.3489
Epoch 75, training loss 1.3301
model finetuning
step  0


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  20
step  40
step  60
step  80
[52.75       53.33333333 56.16666667 56.         57.75       57.83333333
 58.75       58.08333333 59.58333333 60.33333333]
model KNN
step  0
step  20
step  40
step  60
step  80
68.83333333333334


In [23]:
run_BA(4)

model training
Epoch 0, training loss 2.2035
Epoch 25, training loss 1.0298
Epoch 50, training loss 1.0233
Epoch 75, training loss 0.9785
model finetuning


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  0
step  20
step  40
step  60
step  80
[51.75       52.75       56.66666667 59.5        60.91666667 64.
 65.5        67.66666667 70.41666667 72.75      ]
model KNN
step  0
step  20
step  40
step  60
step  80
52.25


In [24]:
run_BA(5)

model training
Epoch 0, training loss 2.6934
Epoch 25, training loss 1.3136
Epoch 50, training loss 1.2073
Epoch 75, training loss 1.1753
model finetuning


/Users/kexinhuang/torch-env/lib/python3.7/site-packages/ipykernel_launcher.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


step  0
step  20
step  40
step  60
step  80
[45.         47.         48.25       50.75       54.91666667 59.33333333
 62.75       64.         66.41666667 68.66666667]
model KNN
step  0
step  20
step  40
step  60
step  80
98.49999999999999
